In [1]:
from bs4 import BeautifulSoup
from pandas import ExcelWriter
from pandas import ExcelFile
from html.parser import HTMLParser
from html.entities import name2codepoint

import requests
import pandas as pd
import os
import datetime

#Valor da hora para utilizacao nas pesquisas
now = datetime.datetime.now()

#Variáveis utilizadas
volume = 50000

#Configura Diretório
os.chdir('C:\\Users\\filip\\Documents\\magic_formula')

#Importa tabela de Ticker que deve estar no diretório de Trabalho
index_table = pd.read_excel('TICKERS.xlsx', sheet_name='Sheet1')
index_table.head(5)

#Cria tabela para guardar indicadores das Empresas
tabela_inicial = pd.DataFrame([],columns = ['TICKER','NOME_EMPRESA','TIPO','SETOR','SUBSETOR','DATA_ULT_BAL','NUM_ACOES','VOL_MEDIO','COTACAO','VALOR_MERCADO','VALOR_FIRMA','P/L','P/VP','PSR','DIV_YIELD','P/EBIT','CRES_REC_5A','LPA','VPA','MARG_BRUTA','MARG_EBIT','MARG_LIQUIDA','EBIT_ATIVO','ROIC','ROE','EV/EBIT','LIQUIDEZ_CORR','DIV_BR_PATRIM','ATIVO', 'DISPONIBILIDADES', 'ATIVO_CIRCULANTE', 'DIV_BRUTA', 'DIV_LIQUIDA', 'PATRIM_LIQUIDO','REC_LIQ_ANU','LUCRO_LIQ_ANU','EBIT_ANU','REC_LIQ_TRI','EBIT_TRI','LUCRO_LIQ_TRI'])
tabela_inicial

#Classe para buscar as informações de text das tags HTML
class MyHTMLParserData(HTMLParser):
    def handle_data(self, data):
        self.valor = data

#Instancia Classe criada acima
parserData = MyHTMLParserData()

#Classe para tratamento do registro e inclusão dos dados na tabela tabela_inicial
class finance_table():
    def init_table(self, ticker, row_number):
        tabela_inicial.loc[row_number] = [ticker,'','','','',0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
        
    def update_table(self, row_number, column_name, column_value):
        if column_name == 'TICKER' or column_name == 'NOME_EMPRESA' or column_name == 'TIPO' or column_name == 'SETOR' or column_name == 'SUBSETOR' or column_name == 'DATA_ULT_BAL':
            tabela_inicial.loc[[row_number],[column_name]] = column_value
        elif column_name == 'DATA_ULT_BAL':
            format_str = '%d/%m/%Y'
            datetime_obj = datetime.datetime.strptime(column_value, format_str)
            tabela_inicial.loc[[row_number],[column_name]] = datetime_obj.date()
        else:
            self.ajusted_value = column_value.replace(".", "")
            self.ajusted_value = self.ajusted_value.replace(" ", "")
            self.ajusted_value = self.ajusted_value.replace("\n", "")
            self.ajusted_value = self.ajusted_value.replace(",", ".")
            self.ajusted_value = self.ajusted_value.replace("%", "")
            
            if self.ajusted_value.find("-") != -1 and len(self.ajusted_value) == 1:
                self.ajusted_value = 0
            
            self.ajusted_float = float(self.ajusted_value)
            
            tabela_inicial.loc[[row_number],[column_name]] = self.ajusted_float
            
#Classe analisar acao
class Conectar():
    def __init__(self, site):
        print("Connect: " + site)
        r  = requests.get(site, headers={'User-Agent': 'Mozilla/5.0'})
        self.data = r.text
        
   #Leitura das tags do site Fundamentus para cada empresa da tabela index_table (tickers)
index = 0
for index, row in index_table.iterrows():
    print("ticker: ",index_table.TICKER[index].lower(), index)

    con = Conectar("http://www.fundamentus.com.br/detalhes.php?papel=" + index_table.TICKER[index].lower())

    soup = BeautifulSoup(con.data)
    table = soup.find_all('table',class_='w728')

    lista = []
    row_td = []
    lista_pai = []
    i = 0

    for table_td in table:
        lista_pai = table_td.find_all('td')
        for row_td in lista_pai:
            lista.append(row_td)

    anual = 0        
    data_ult = 0
    ft = finance_table()
    ft.init_table(index_table.TICKER[index],index)

    for i in range(len(lista)):

        if str(lista[i]).find(">Últ balanço processado<")>0 and data_ult == 0: 
            parserData.feed(str(lista[i+1]))
            #print("Últ balanço processado: ",parserData.valor)
            ft.update_table(index,'DATA_ULT_BAL',parserData.valor)
            data_ult = 1

        elif str(lista[i]).find(">Cotação<")>0: 
            parserData.feed(str(lista[i+1]))
            #print("Cotação: ",parserData.valor)
            ft.update_table(index,'COTACAO',parserData.valor)

        elif str(lista[i]).find(">Tipo<")>0: 
            parserData.feed(str(lista[i+1]))
            #print("Tipo: ",parserData.valor)
            ft.update_table(index,'TIPO',parserData.valor)

        elif str(lista[i]).find(">Empresa<")>0: 
            parserData.feed(str(lista[i+1]))
            #print("Empresa: ",parserData.valor)
            ft.update_table(index,'NOME_EMPRESA',parserData.valor)

        elif str(lista[i]).find(">Setor<")>0: 
            parserData.feed(str(lista[i+1]))
            #print("Setor: ",parserData.valor)
            ft.update_table(index,'SETOR',parserData.valor)

        elif str(lista[i]).find(">Subsetor<")>0: 
            parserData.feed(str(lista[i+1]))
            #print("Subsetor: ",parserData.valor)
            ft.update_table(index,'SUBSETOR',parserData.valor)

        elif str(lista[i]).find(">Últ balanço processado<")>0: 
            parserData.feed(str(lista[i+1]))
            #print("Últ balanço processado: ",parserData.valor)
            ft.update_table(index,'DATA_ULT_BAL',parserData.valor)

        elif str(lista[i]).find(">Vol $ méd (2m)<")>0: 
            parserData.feed(str(lista[i+1]))
            #print("Vol $ méd (2m): ",parserData.valor)
            ft.update_table(index,'VOL_MEDIO',parserData.valor)

        elif str(lista[i]).find(">Valor de mercado<")>0: 
            parserData.feed(str(lista[i+1]))
            #print("Valor de mercado: ",parserData.valor)
            ft.update_table(index,'VALOR_MERCADO',parserData.valor)

        elif str(lista[i]).find(">Valor da firma<")>0: 
            parserData.feed(str(lista[i+1]))
            #print("Valor da firma: ",parserData.valor)
            ft.update_table(index,'VALOR_FIRMA',parserData.valor)            

        elif str(lista[i]).find(">Nro. Ações<")>0: 
            parserData.feed(str(lista[i+1]))
            #print("Nro. Ações: ",parserData.valor)
            ft.update_table(index,'NUM_ACOES',parserData.valor)

        elif str(lista[i]).find(">P/L<")>0: 
            parserData.feed(str(lista[i+1]))
            #print("P/L: ",parserData.valor)
            ft.update_table(index,'P/L',parserData.valor)

        elif str(lista[i]).find(">LPA<")>0: 
            parserData.feed(str(lista[i+1]))
            #print("LPA: ",parserData.valor)
            ft.update_table(index,'LPA',parserData.valor)

        elif str(lista[i]).find(">P/VP<")>0: 
            parserData.feed(str(lista[i+1]))
            #print("P/VP: ",parserData.valor)
            ft.update_table(index,'P/VP',parserData.valor)

        elif str(lista[i]).find(">VPA<")>0: 
            parserData.feed(str(lista[i+1]))
            #print("VPA: ",parserData.valor)
            ft.update_table(index,'VPA',parserData.valor)

        elif str(lista[i]).find(">P/EBIT<")>0: 
            parserData.feed(str(lista[i+1]))
            #print("P/EBIT: ",parserData.valor)
            ft.update_table(index,'P/EBIT',parserData.valor)

        elif str(lista[i]).find(">Marg. Bruta<")>0: 
            parserData.feed(str(lista[i+1]))
            #print("Marg. Bruta: ",parserData.valor)
            ft.update_table(index,'MARG_BRUTA',parserData.valor) 

        elif str(lista[i]).find(">PSR<")>0: 
            parserData.feed(str(lista[i+1]))
            #print("PSR: ",parserData.valor)
            ft.update_table(index,'PSR',parserData.valor) 

        elif str(lista[i]).find(">Marg. EBIT<")>0: 
            parserData.feed(str(lista[i+1]))
            #print("Marg. EBIT: ",parserData.valor)
            ft.update_table(index,'MARG_EBIT',parserData.valor) 

        elif str(lista[i]).find(">Marg. Líquida<")>0: 
            parserData.feed(str(lista[i+1]))
            #print("Marg. Líquida: ",parserData.valor)
            ft.update_table(index,'MARG_LIQUIDA',parserData.valor) 

        elif str(lista[i]).find(">EBIT / Ativo<")>0: 
            parserData.feed(str(lista[i+1]))
            #print("EBIT / Ativo: ",parserData.valor)
            ft.update_table(index,'EBIT_ATIVO',parserData.valor) 

        elif str(lista[i]).find(">ROIC<")>0: 
            parserData.feed(str(lista[i+1]))
            #print("ROIC: ",parserData.valor)
            ft.update_table(index,'ROIC',parserData.valor)           

        elif str(lista[i]).find(">Div. Yield<")>0: 
            parserData.feed(str(lista[i+1]))
            #print("Div. Yield: ",parserData.valor)
            ft.update_table(index,'DIV_YIELD',parserData.valor) 

        elif str(lista[i]).find(">ROE<")>0: 
            parserData.feed(str(lista[i+1]))
            #print("ROE: ",parserData.valor)
            ft.update_table(index,'ROE',parserData.valor) 

        elif str(lista[i]).find(">EV / EBIT<")>0: 
            parserData.feed(str(lista[i+1]))
            #print("EV / EBIT: ",parserData.valor)
            ft.update_table(index,'EV/EBIT',parserData.valor) 

        elif str(lista[i]).find(">Liquidez Corr<")>0: 
            parserData.feed(str(lista[i+1]))
            #print("Liquidez Corr: ",parserData.valor)
            ft.update_table(index,'LIQUIDEZ_CORR',parserData.valor)   

        elif str(lista[i]).find(">Div Br/ Patrim<")>0: 
            parserData.feed(str(lista[i+1]))
            #print("Div Br/ Patrim: ",parserData.valor)
            ft.update_table(index,'DIV_BR_PATRIM',parserData.valor)  

        elif str(lista[i]).find(">Cres. Rec (5a)<")>0: 
            parserData.feed(str(lista[i+1]))
            #print("Cres. Rec (5a): ",parserData.valor)
            ft.update_table(index,'CRES_REC_5A',parserData.valor)  

        elif str(lista[i]).find(">Ativo<")>0: 
            parserData.feed(str(lista[i+1]))
            #print("Ativo: ",parserData.valor)
            ft.update_table(index,'ATIVO',parserData.valor)  

        elif str(lista[i]).find(">Disponibilidades<")>0: 
            parserData.feed(str(lista[i+1]))
            #print("Disponibilidades: ",parserData.valor)
            ft.update_table(index,'DISPONIBILIDADES',parserData.valor)  

        elif str(lista[i]).find(">Ativo Circulante<")>0: 
            parserData.feed(str(lista[i+1]))
            #print("Ativo Circulante: ",parserData.valor)
            ft.update_table(index,'ATIVO_CIRCULANTE',parserData.valor)    

        elif str(lista[i]).find(">Dív. Bruta<")>0: 
            parserData.feed(str(lista[i+1]))
            #print("Dív. Bruta: ",parserData.valor)
            ft.update_table(index,'DIV_BRUTA',parserData.valor)  

        elif str(lista[i]).find(">Dív. Líquida<")>0: 
            parserData.feed(str(lista[i+1]))
            #print("Dív. Líquida: ",parserData.valor)
            ft.update_table(index,'DIV_LIQUIDA',parserData.valor)  

        elif str(lista[i]).find(">Patrim. Líq<")>0: 
            parserData.feed(str(lista[i+1]))
            #print("Patrim. Líq: ",parserData.valor)
            ft.update_table(index,'PATRIM_LIQUIDO',parserData.valor)              

        elif str(lista[i]).find(">Receita Líquida<")>0:
            parserData.feed(str(lista[i+1]))
            if anual != 1:
                anual = 1
                #print("Receita Líquida Anual: ",parserData.valor)
                ft.update_table(index,'REC_LIQ_ANU',parserData.valor)
            else:
                #print("Receita líquida Trimestral: ",parserData.valor)
                ft.update_table(index,'REC_LIQ_TRI',parserData.valor)
                anual = 0

        elif str(lista[i]).find(">EBIT<")>0:
            parserData.feed(str(lista[i+1]))
            if anual != 1:
                anual = 1
                #print("EBIT Anual: ",parserData.valor)
                ft.update_table(index,'EBIT_ANU',parserData.valor)            
            else:
                #print("EBIT Trimestral: ",parserData.valor)
                ft.update_table(index,'EBIT_TRI',parserData.valor)        
                anual = 0 

        elif str(lista[i]).find(">Lucro Líquido<")>0:
            parserData.feed(str(lista[i+1]))
            if anual != 1:
                anual = 1
                #print("Lucro Líquido Anual: ",parserData.valor)
                ft.update_table(index,'LUCRO_LIQ_ANU',parserData.valor)            
            else:
                #print("Lucro Líquido Trimestral: ",parserData.valor)
                ft.update_table(index,'LUCRO_LIQ_TRI',parserData.valor)        
                anual = 0
                
#Retira Valores zerados para os indicadores que iremos utilizar
tabela_szero = tabela_inicial.query("VALOR_FIRMA > 0 and ROIC > 0 and EBIT_ANU > 0")

#Volume maior do que 50000
tabela_volume = tabela_szero[tabela_szero['VOL_MEDIO'] > volume]

#Filtra somente balanços do ano atual e passado para não pegar sujeira como empresas que não estão mais na bolsa
tabela_year = tabela_volume[(tabela_volume['DATA_ULT_BAL'].str.contains(str(now.year-1)))|(tabela_volume['DATA_ULT_BAL'].str.contains( str(now.year)))]

#Ordenando por EV/EBIT
tabela_ord1 = tabela_year.sort_values('EV/EBIT',ascending=True)

#Rankeando EV/EBIT
index = 0
count = 0
for index, row in tabela_ord1.iterrows():
    count = count + 1
    tabela_ord1.loc[index,'ORD_EVEBIT'] = count
    
#Ordenando por ROIC
tabela_ord2 = tabela_ord1.sort_values('ROIC',ascending=False)

#Rankeando ROIC
index = 0
count = 0
for index, row in tabela_ord2.iterrows():
    count = count + 1
    tabela_ord2.loc[index,'ORD_ROIC'] = count
    
#Somando os 2 rankeamentos
index = 0
count = 0
for index, row in tabela_ord2.iterrows():
    count = count + 1
    tabela_ord2.loc[index,'RANKING'] = tabela_ord2.loc[index,'ORD_EVEBIT'] + tabela_ord2.loc[index,'ORD_ROIC']
    
#Ordena tabela de Rankeamento
tabela_ord_rank = tabela_ord2.sort_values('RANKING',ascending=True)

#Exporta o Excel com a formula mágica
tabela_ord_rank.to_excel("dados_balanco_calculado_"+now.strftime("%Y-%m-%d")+".xlsx")

tabela_inicial.to_excel("tabela_fundamentus.xlsx")

ticker:  aalr3 0
Connect: http://www.fundamentus.com.br/detalhes.php?papel=aalr3
ticker:  abcb3 1
Connect: http://www.fundamentus.com.br/detalhes.php?papel=abcb3
ticker:  abcb4 2
Connect: http://www.fundamentus.com.br/detalhes.php?papel=abcb4
ticker:  abev3 3
Connect: http://www.fundamentus.com.br/detalhes.php?papel=abev3
ticker:  abre3 4
Connect: http://www.fundamentus.com.br/detalhes.php?papel=abre3
ticker:  abya3 5
Connect: http://www.fundamentus.com.br/detalhes.php?papel=abya3
ticker:  aces3 6
Connect: http://www.fundamentus.com.br/detalhes.php?papel=aces3
ticker:  aces4 7
Connect: http://www.fundamentus.com.br/detalhes.php?papel=aces4
ticker:  adhm3 8
Connect: http://www.fundamentus.com.br/detalhes.php?papel=adhm3
ticker:  aedu11 9
Connect: http://www.fundamentus.com.br/detalhes.php?papel=aedu11
ticker:  aedu3 10
Connect: http://www.fundamentus.com.br/detalhes.php?papel=aedu3
ticker:  aelp3 11
Connect: http://www.fundamentus.com.br/detalhes.php?papel=aelp3
ticker:  aesl3 12
Connec

ticker:  bicb3 100
Connect: http://www.fundamentus.com.br/detalhes.php?papel=bicb3
ticker:  bicb4 101
Connect: http://www.fundamentus.com.br/detalhes.php?papel=bicb4
ticker:  bidi11 102
Connect: http://www.fundamentus.com.br/detalhes.php?papel=bidi11
ticker:  bidi3 103
Connect: http://www.fundamentus.com.br/detalhes.php?papel=bidi3
ticker:  bidi4 104
Connect: http://www.fundamentus.com.br/detalhes.php?papel=bidi4
ticker:  biom3 105
Connect: http://www.fundamentus.com.br/detalhes.php?papel=biom3
ticker:  biom4 106
Connect: http://www.fundamentus.com.br/detalhes.php?papel=biom4
ticker:  bisa3 107
Connect: http://www.fundamentus.com.br/detalhes.php?papel=bisa3
ticker:  bkbr3 108
Connect: http://www.fundamentus.com.br/detalhes.php?papel=bkbr3
ticker:  bmeb3 109
Connect: http://www.fundamentus.com.br/detalhes.php?papel=bmeb3
ticker:  bmeb4 110
Connect: http://www.fundamentus.com.br/detalhes.php?papel=bmeb4
ticker:  bmef3 111
Connect: http://www.fundamentus.com.br/detalhes.php?papel=bmef3
ti

ticker:  ceeb5 199
Connect: http://www.fundamentus.com.br/detalhes.php?papel=ceeb5
ticker:  ceed3 200
Connect: http://www.fundamentus.com.br/detalhes.php?papel=ceed3
ticker:  ceed4 201
Connect: http://www.fundamentus.com.br/detalhes.php?papel=ceed4
ticker:  cegr3 202
Connect: http://www.fundamentus.com.br/detalhes.php?papel=cegr3
ticker:  celm3 203
Connect: http://www.fundamentus.com.br/detalhes.php?papel=celm3
ticker:  celp5 204
Connect: http://www.fundamentus.com.br/detalhes.php?papel=celp5
ticker:  celp7 205
Connect: http://www.fundamentus.com.br/detalhes.php?papel=celp7
ticker:  cepe3 206
Connect: http://www.fundamentus.com.br/detalhes.php?papel=cepe3
ticker:  cepe5 207
Connect: http://www.fundamentus.com.br/detalhes.php?papel=cepe5
ticker:  cepe6 208
Connect: http://www.fundamentus.com.br/detalhes.php?papel=cepe6
ticker:  cesp3 209
Connect: http://www.fundamentus.com.br/detalhes.php?papel=cesp3
ticker:  cesp4 210
Connect: http://www.fundamentus.com.br/detalhes.php?papel=cesp4
tick

ticker:  dfva3 298
Connect: http://www.fundamentus.com.br/detalhes.php?papel=dfva3
ticker:  dfva4 299
Connect: http://www.fundamentus.com.br/detalhes.php?papel=dfva4
ticker:  dhbi3 300
Connect: http://www.fundamentus.com.br/detalhes.php?papel=dhbi3
ticker:  dhbi4 301
Connect: http://www.fundamentus.com.br/detalhes.php?papel=dhbi4
ticker:  dirr3 302
Connect: http://www.fundamentus.com.br/detalhes.php?papel=dirr3
ticker:  djon4 303
Connect: http://www.fundamentus.com.br/detalhes.php?papel=djon4
ticker:  dmmo3 304
Connect: http://www.fundamentus.com.br/detalhes.php?papel=dmmo3
ticker:  doca3 305
Connect: http://www.fundamentus.com.br/detalhes.php?papel=doca3
ticker:  doca4 306
Connect: http://www.fundamentus.com.br/detalhes.php?papel=doca4
ticker:  dohl3 307
Connect: http://www.fundamentus.com.br/detalhes.php?papel=dohl3
ticker:  dohl4 308
Connect: http://www.fundamentus.com.br/detalhes.php?papel=dohl4
ticker:  dppi3 309
Connect: http://www.fundamentus.com.br/detalhes.php?papel=dppi3
tick

ticker:  flry3 397
Connect: http://www.fundamentus.com.br/detalhes.php?papel=flry3
ticker:  fras3 398
Connect: http://www.fundamentus.com.br/detalhes.php?papel=fras3
ticker:  fras4 399
Connect: http://www.fundamentus.com.br/detalhes.php?papel=fras4
ticker:  frio3 400
Connect: http://www.fundamentus.com.br/detalhes.php?papel=frio3
ticker:  frta3 401
Connect: http://www.fundamentus.com.br/detalhes.php?papel=frta3
ticker:  ftrx3 402
Connect: http://www.fundamentus.com.br/detalhes.php?papel=ftrx3
ticker:  ftrx4 403
Connect: http://www.fundamentus.com.br/detalhes.php?papel=ftrx4
ticker:  gafp3 404
Connect: http://www.fundamentus.com.br/detalhes.php?papel=gafp3
ticker:  gafp4 405
Connect: http://www.fundamentus.com.br/detalhes.php?papel=gafp4
ticker:  galo3 406
Connect: http://www.fundamentus.com.br/detalhes.php?papel=galo3
ticker:  galo4 407
Connect: http://www.fundamentus.com.br/detalhes.php?papel=galo4
ticker:  gazo3 408
Connect: http://www.fundamentus.com.br/detalhes.php?papel=gazo3
tick

ticker:  lcam3 496
Connect: http://www.fundamentus.com.br/detalhes.php?papel=lcam3
ticker:  lcsa3 497
Connect: http://www.fundamentus.com.br/detalhes.php?papel=lcsa3
ticker:  lcsa4 498
Connect: http://www.fundamentus.com.br/detalhes.php?papel=lcsa4
ticker:  leco3 499
Connect: http://www.fundamentus.com.br/detalhes.php?papel=leco3
ticker:  leco4 500
Connect: http://www.fundamentus.com.br/detalhes.php?papel=leco4
ticker:  leto3 501
Connect: http://www.fundamentus.com.br/detalhes.php?papel=leto3
ticker:  leto5 502
Connect: http://www.fundamentus.com.br/detalhes.php?papel=leto5
ticker:  leve3 503
Connect: http://www.fundamentus.com.br/detalhes.php?papel=leve3
ticker:  leve4 504
Connect: http://www.fundamentus.com.br/detalhes.php?papel=leve4
ticker:  lffe3 505
Connect: http://www.fundamentus.com.br/detalhes.php?papel=lffe3
ticker:  lffe4 506
Connect: http://www.fundamentus.com.br/detalhes.php?papel=lffe4
ticker:  lglo4 507
Connect: http://www.fundamentus.com.br/detalhes.php?papel=lglo4
tick

ticker:  palf5 595
Connect: http://www.fundamentus.com.br/detalhes.php?papel=palf5
ticker:  parc3 596
Connect: http://www.fundamentus.com.br/detalhes.php?papel=parc3
ticker:  pard3 597
Connect: http://www.fundamentus.com.br/detalhes.php?papel=pard3
ticker:  pati3 598
Connect: http://www.fundamentus.com.br/detalhes.php?papel=pati3
ticker:  pati4 599
Connect: http://www.fundamentus.com.br/detalhes.php?papel=pati4
ticker:  pcar3 600
Connect: http://www.fundamentus.com.br/detalhes.php?papel=pcar3
ticker:  pcar4 601
Connect: http://www.fundamentus.com.br/detalhes.php?papel=pcar4
ticker:  pcar5 602
Connect: http://www.fundamentus.com.br/detalhes.php?papel=pcar5
ticker:  pdgr3 603
Connect: http://www.fundamentus.com.br/detalhes.php?papel=pdgr3
ticker:  peab3 604
Connect: http://www.fundamentus.com.br/detalhes.php?papel=peab3
ticker:  peab4 605
Connect: http://www.fundamentus.com.br/detalhes.php?papel=peab4
ticker:  pefx3 606
Connect: http://www.fundamentus.com.br/detalhes.php?papel=pefx3
tick

ticker:  rpmg4 694
Connect: http://www.fundamentus.com.br/detalhes.php?papel=rpmg4
ticker:  rpsa4 695
Connect: http://www.fundamentus.com.br/detalhes.php?papel=rpsa4
ticker:  rsid3 696
Connect: http://www.fundamentus.com.br/detalhes.php?papel=rsid3
ticker:  rsip3 697
Connect: http://www.fundamentus.com.br/detalhes.php?papel=rsip3
ticker:  rsip4 698
Connect: http://www.fundamentus.com.br/detalhes.php?papel=rsip4
ticker:  rsul4 699
Connect: http://www.fundamentus.com.br/detalhes.php?papel=rsul4
ticker:  rumo3 700
Connect: http://www.fundamentus.com.br/detalhes.php?papel=rumo3
ticker:  salm3 701
Connect: http://www.fundamentus.com.br/detalhes.php?papel=salm3
ticker:  salm4 702
Connect: http://www.fundamentus.com.br/detalhes.php?papel=salm4
ticker:  sanb11 703
Connect: http://www.fundamentus.com.br/detalhes.php?papel=sanb11
ticker:  sanb3 704
Connect: http://www.fundamentus.com.br/detalhes.php?papel=sanb3
ticker:  sanb4 705
Connect: http://www.fundamentus.com.br/detalhes.php?papel=sanb4
ti

ticker:  temp3 793
Connect: http://www.fundamentus.com.br/detalhes.php?papel=temp3
ticker:  tend3 794
Connect: http://www.fundamentus.com.br/detalhes.php?papel=tend3
ticker:  tene5 795
Connect: http://www.fundamentus.com.br/detalhes.php?papel=tene5
ticker:  tene7 796
Connect: http://www.fundamentus.com.br/detalhes.php?papel=tene7
ticker:  teri3 797
Connect: http://www.fundamentus.com.br/detalhes.php?papel=teri3
ticker:  tesa3 798
Connect: http://www.fundamentus.com.br/detalhes.php?papel=tesa3
ticker:  tgma3 799
Connect: http://www.fundamentus.com.br/detalhes.php?papel=tgma3
ticker:  tibr3 800
Connect: http://www.fundamentus.com.br/detalhes.php?papel=tibr3
ticker:  tibr5 801
Connect: http://www.fundamentus.com.br/detalhes.php?papel=tibr5
ticker:  tibr6 802
Connect: http://www.fundamentus.com.br/detalhes.php?papel=tibr6
ticker:  tiet11 803
Connect: http://www.fundamentus.com.br/detalhes.php?papel=tiet11
ticker:  tiet3 804
Connect: http://www.fundamentus.com.br/detalhes.php?papel=tiet3
ti

ticker:  vvax3 892
Connect: http://www.fundamentus.com.br/detalhes.php?papel=vvax3
ticker:  vvax4 893
Connect: http://www.fundamentus.com.br/detalhes.php?papel=vvax4
ticker:  wege3 894
Connect: http://www.fundamentus.com.br/detalhes.php?papel=wege3
ticker:  wege4 895
Connect: http://www.fundamentus.com.br/detalhes.php?papel=wege4
ticker:  whrl3 896
Connect: http://www.fundamentus.com.br/detalhes.php?papel=whrl3
ticker:  whrl4 897
Connect: http://www.fundamentus.com.br/detalhes.php?papel=whrl4
ticker:  wisa3 898
Connect: http://www.fundamentus.com.br/detalhes.php?papel=wisa3
ticker:  wisa4 899
Connect: http://www.fundamentus.com.br/detalhes.php?papel=wisa4
ticker:  wizs3 900
Connect: http://www.fundamentus.com.br/detalhes.php?papel=wizs3
ticker:  wlmm3 901
Connect: http://www.fundamentus.com.br/detalhes.php?papel=wlmm3
ticker:  wlmm4 902
Connect: http://www.fundamentus.com.br/detalhes.php?papel=wlmm4
ticker:  wmby3 903
Connect: http://www.fundamentus.com.br/detalhes.php?papel=wmby3
tick